In [1]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
from PIL import Image
import torch.utils.data as data

In [2]:
if torch.cuda.is_available(): 
    device='cuda'
else:
    device='cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

print(torch.cuda.is_available())

True


In [66]:
def make_file_list():
    train_img_list = list()
    for idx in range(1, 4001):
        img_path = "./training_set/cats/cat." + str(idx) + '.jpg'
        train_img_list.append(img_path)
        
        img_path = "./training_set/dogs/dog." + str(idx) + '.jpg'
        train_img_list.append(img_path)
        
    return train_img_list

In [67]:
class ImageTransform():
    
    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            transforms.RandomResizedCrop(224)
        ])
        
    def __call__(self, img):
        return self.data_transform(img)

In [68]:
class ImageDataset(data.Dataset):
    
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path[15:16]

        if label == "c":
            label = 0
        elif label == "d":
            label = 1

       
        return img_transformed, label

In [69]:
train_list = make_file_list()
mean = (0.5,)
std = (0.3,)

In [70]:
train_dataset = ImageDataset(file_list = train_list, transform=ImageTransform(mean, std))

In [71]:
train_dataloader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [72]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 5),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, 5),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, 5),
            torch.nn.Dropout(0.2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))


        self.fc1 = torch.nn.Linear(21 * 21 * 256, 512, bias=True)
        self.fc2 = torch.nn.Linear(512, 128, bias=True)
        self.fc3 = torch.nn.Linear(128, 2, bias=True)

        torch.nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [73]:
model = CNN().to(device)